# Dependence of the condition number $cond(\hat{\mathcal{G}}\hat{\mathcal{F}})$ on c.m. energy spread

In [ ]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
from PyISR import ISRSolverSLE
from ROOT import TFile
import matplotlib
import matplotlib.pyplot as plt
import mplhep as hep
from tqdm import notebook
plt.style.use(hep.style.CMS)
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.sans-serif'] = ['Tahoma', 'DejaVu Sans',
                                          'Lucida Grande', 'Verdana']

Remove scrolling:

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline

Function for reading visible cross section data:

In [ ]:
def readVCS(path):
    fl = TFile.Open(path, "read")
    # Getting a pointer to the visible cross section in the form of TGraphErrors
    gvcs = fl.Get('vcs')
    # Number of c.m. energy points
    n = gvcs.GetN()
    # Reading c.m. energy array
    energy = np.frombuffer(gvcs.GetX(), dtype=np.float64, count=n)
    # Reading array of c.m. energy errors
    energyErr = np.frombuffer(gvcs.GetEX(), dtype=np.float64, count=n)
    # Reading visible cross section array
    vcs = np.frombuffer(gvcs.GetY(), dtype=np.float64, count=n)
    # Reading array of visible cross section errors
    vcsErr = np.frombuffer(gvcs.GetEY(), dtype=np.float64, count=n)
    fl.Close()
    return energy, vcs, energyErr, vcsErr

Function for obtaining numerical solution:

In [ ]:
def solve(energy, vcs, energyErr, vcsErr, 
          enabled_energy_spread=False, 
          threshold_energy=0.827, 
          interp=None,
          efficiency=lambda x, en: 1.0):
    n = energy.shape[0]
    solver = ISRSolverSLE(n, energy, vcs, 
                          energyErr, vcsErr, 
                          threshold_energy, efficiency,
                          enabled_energy_spread)
    if type(interp) == str:
        with open(interp, 'r') as jfl:
            settings = json.load(jfl)
        
        print('Interpolation settings:')
        print(settings)
        solver.set_interp_settings(settings)
    elif type(interp) == list:
        print('Interpolation settings:')
        print(interp)
        solver.set_interp_settings(interp)
    
    solver.solve()
    return solver

In [ ]:
# Number c.m. energy spread points
npt = 100
# Function for preparing dependence of the condition number of c.m. energy spread
def makeCondNumData(input_data):
    pbar = notebook.tqdm(total=npt, position=input_data['position'])
    solver = solve(*readVCS(input_data['input_path']))
    pbar.update()
    enspread = np.linspace(0.,0.014, npt)
    condnum = [solver.condnum_eval(),]
    solver.energy_spread_enabled = True
    for i in range(1, enspread.shape[0]):
        solver.reset_ecm_err(enspread[i] * np.ones(solver.n))
        solver.eval_equation_matrix()
        condnum.append(solver.condnum_eval())
        pbar.update()
    
    condnum = np.array(condnum)
    return enspread, condnum
    

## Equally spaced c.m. energy points

In [ ]:
input_data = [
    {'input_path' : 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread.root',
    'label' : r'$50$ equally spaced c.m. energy points',
    'position' : 0},
    {'input_path' : 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_100pts.root',
    'label' : r'$100$ equally spaced c.m. energy points',
    'position' : 1},
]
cndData = map(makeCondNumData, input_data)

In [ ]:
# Function for plotting dependence of the condition number of the c.m. energy spread
def condnumPlot(out_data, in_data):
    fontsize = 24
    f, ax = plt.subplots(figsize=(9, 7))
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    for i, (x, y) in enumerate(out_data):
        ax.plot(x * 1.e+3, y, label=in_data[i]['label'])
        
    ax.set_yscale('log')
    ax.set_xlabel(r'$\sigma_E$ (MeV)', fontsize=fontsize)
    ax.set_ylabel('condition number', fontsize=fontsize)
    ax.legend(fontsize=fontsize, bbox_to_anchor=(1.05, 1))

In [ ]:
condnumPlot(cndData, input_data)

## Highly non-uniform distribution of c.m.energy points

In [ ]:
input_data = [
    {'input_path' : 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_nonuniform.root',
    'label' : r'$50$ non-uniform c.m. energy points',
    'position' : 0}
]
cndData = map(makeCondNumData, input_data)
condnumPlot(cndData, input_data)

In [ ]:
# Opening the file containing the model cross section
fl = TFile.Open('data/born_cs_etapipi_simple_model.root', 'read')
# Cloning the model cross section function
mBCsFcn = fl.Get('f_bcs').Clone()
# Vectorizing the model Born cross section function
mBCsVFcn = np.vectorize(lambda en: mBCsFcn.Eval(en))
# Closing the file
fl.Close()

In [ ]:
def csPlot(solver, title='Cross sections', fontsize=24):
    # Getting c.m. energy array
    ecm = solver.ecm()
    f, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, sharex=True,
                                figsize=(9, 10))
    f.suptitle(title, fontsize=fontsize)
    ax1.tick_params(axis='both', which='major', labelsize=fontsize)
    ecm_dense = np.linspace(np.min(ecm), np.max(ecm), ecm.shape[0] * 20)
    interp = np.vectorize(lambda en: solver.interp_eval(en))(ecm_dense)
    ax0.errorbar(ecm, solver.vcs(), 
                xerr=solver.ecm_err(),
                yerr=solver.vcs_err(), fmt='o',
                markersize=5, capsize=3,
                label='Visible cross section', zorder=0)
    bcs_err = yerr=np.sqrt(np.diag(solver.bcs_cov_matrix()))
    ax0.errorbar(ecm, solver.bcs(), yerr=bcs_err, fmt='o',
                 markersize=5, capsize=3,
                label='Born cross section', zorder=1)
    ax0.plot(ecm_dense, interp, 'b--', label='Interpolation of the Born cross section', zorder=2)
    ax0.plot(ecm_dense, mBCsVFcn(ecm_dense), 'r-', label='Model Born cross section', zorder=3)
    mBCs_at_ecm =  mBCsVFcn(ecm)
    ax1.errorbar(ecm, solver.bcs() / mBCs_at_ecm, yerr=bcs_err / mBCs_at_ecm, fmt='o',
                 markersize=5, capsize=3,
                 label=r'Ratio $\frac{\sigma_{\rm B}}{\sigma^{\rm model}_{\rm B}}$',
            zorder=0)
    ax1.set_xlabel(r'$\sqrt{s}$ (GeV)', fontsize=fontsize)
    ax0.set_ylabel('cross section (nb)', fontsize=fontsize)
    ax0.legend(fontsize=fontsize, bbox_to_anchor=(1.05, 1))

### $\sigma_E = 0\text{ MeV}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_nonuniform.root'
energy, vcs, energyErr, vcsErr = readVCS(input_path)
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=False)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)

### $\sigma_E = 0.2\text{ MeV}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_nonuniform.root'
energy, vcs, _, vcsErr = readVCS(input_path)
energyErr = 2.e-4 * np.ones(energy.shape[0])
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)

### $\sigma_E = 0.8\text{ MeV}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_nonuniform.root'
energy, vcs, _, vcsErr = readVCS(input_path)
energyErr = 8.e-4 * np.ones(energy.shape[0])
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)

### $\sigma_E = 1\text{ MeV}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_nonuniform.root'
energy, vcs, _, vcsErr = readVCS(input_path)
energyErr = 1.e-3 * np.ones(energy.shape[0])
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)

### $\sigma_E = 2\text{ MeV}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_nonuniform.root'
energy, vcs, _, vcsErr = readVCS(input_path)
energyErr = 2.e-3 * np.ones(energy.shape[0])
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)

## The case of two close c.m. energy points. 
The distance between almost all neighboring c.m. energy points is about $16.4\text{ MeV}$, but there are two points with distance $1\text{ MeV}$ 

In [ ]:
input_data = [
    {'input_path' : 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_two_close_points.root',
    'label' : 'case of two close c.m. energy points',
    'position' : 0}
]
cndData = map(makeCondNumData, input_data)
condnumPlot(cndData, input_data)

### $\sigma_E = 0\text{ MeV}$

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_two_close_points.root'
energy, vcs, energyErr, vcsErr = readVCS(input_path)
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=False)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)

### $\sigma_E = 2\text{MeV}$
There is a large scatter of the cross section at close points.

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread_two_close_points.root'
energy, vcs, _, vcsErr = readVCS(input_path)
energyErr = 2.e-3 * np.ones(energy.shape[0])
solver = solve(energy, vcs, energyErr, vcsErr, enabled_energy_spread=True)
print('Condition number: {}'.format(solver.condnum_eval()))
csPlot(solver)